automated way of getting the valid characters of each language

using: https://unicode-org.github.io/cldr-staging/charts/latest/summary

In [36]:
from bs4 import BeautifulSoup
import requests
import json
import re
import glob

base_url = "https://unicode-org.github.io/cldr-staging/charts/latest/summary/"

data_dir = "../webapp/data/"
language_codes = [f.split("/")[-1] for f in glob.glob(f"{data_dir}/languages/*")]

# for each language, get the html
language_characters = {}
for language_code in language_codes:
    try:
        url = f"{base_url}{language_code}.html"
        print(f"Getting {url}")
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")

        # get the table
        table = soup.find(
            "table", {"border": "1", "cellpadding": "2", "cellspacing": "0"}
        )
        # find index of table header with text "Native"
        header_index = [
            i for i, th in enumerate(table.find_all("th")) if "Native" in th.text
        ][0]
        text = table.find_all("td")[header_index].text

        # remove [ and ]
        text = text.replace("[", "")
        text = text.replace("]", "")

        # split by space
        text = text.split(" ")
        print(f"{language_code}: {text}")

        # store
        language_characters[language_code] = text
        # remove characters longer than 1
        language_characters[language_code] = [
            char for char in language_characters[language_code] if len(char) == 1
        ]
    except:
        print(f"{language_code} not found")


# store to json in /out/
with open(f"out/languages_characters.json", "w", encoding="utf-8") as f:
    json.dump(language_characters, f, ensure_ascii=False, indent=4)

Getting https://unicode-org.github.io/cldr-staging/charts/latest/summary/ie.html
ie not found
Getting https://unicode-org.github.io/cldr-staging/charts/latest/summary/ko.html
ko: ['가-힣']
Getting https://unicode-org.github.io/cldr-staging/charts/latest/summary/ne.html
ne: ['\\u093C', '\\u0901', '\\u0902', 'ः', 'ॐ', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ऌ', 'ऍ', 'ए', 'ऐ', 'ऑ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'ळ', 'व', 'श', 'ष', 'स', 'ह', 'ऽ', 'ा', 'ि', 'ी', '\\u0941', '\\u0942', '\\u0943', '\\u0944', '\\u0945', '\\u0947', '\\u0948', 'ॉ', 'ो', 'ौ', '\\u094D']
Getting https://unicode-org.github.io/cldr-staging/charts/latest/summary/sl.html
sl: ['a', 'b', 'c', 'č', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 'š', 't', 'u', 'v', 'z', 'ž']
Getting https://unicode-org.github.io/cldr-staging/charts/latest/summary/nl.html
nl: ['a', 'á', 'ä', 'b', 'c',

In [37]:
# store each characterset in '../webapp/data/languages/<language_code>/<language_code>_characters.txt'
for k, v in language_characters.items():
    if k in ["en"]:
        continue
    # if theres less than 10 characters, skip (like korean)
    if len(v) < 10:
        continue
    with open(f"{data_dir}/languages/{k}/{k}_characters.txt", "w") as f:
        for char in v:
            f.write(f"{char}\n")